In [ ]:
!pip install Modelbit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 4.2 MB/s eta 0:00:00


# IMPORTS Y FUNCIONES

In [ ]:
import modelbit
import xgboost as xgb
import pandas as pd
import numpy as np


In [ ]:
def calculate_lags_and_means(df, day):
    lags = [1, 2, 3, 4, 5, 6, 7, 10, 12, 15]
    df_lag = df[(df['d'] <= day) & (df['d'] >= day - max(lags))].copy()

    for lag in lags:
        df_lag['sold_lag_' + str(lag)] = (
            df_lag.groupby(['item_id', 'dept_id', 'cat_id'], as_index=False)['sold']
            .shift(lag).astype(np.float16)
        )
        # Asegura la alineación de índices entre df y df_lag antes de asignar
        df.loc[df['d'] == day, 'sold_lag_' + str(lag)] = df_lag.loc[df_lag['d'] == day, 'sold_lag_' + str(lag)].values

    # Calcular medias rodantes y expansivas
    df_means = df[(df['d'] <= day) & (df['d'] >= day - 28)].copy()
    df_means['rolling_sold_mean'] = (
        df_means.groupby(['item_id', 'dept_id', 'cat_id'])['sold']
        .transform(lambda x: x.rolling(window=7, min_periods=1).mean()).astype(np.float16)
    )

    df_means['rolling_sold_mean_14'] = (
        df_means.groupby(['item_id', 'dept_id', 'cat_id'])['sold']
        .transform(lambda x: x.rolling(window=14, min_periods=1).mean()).astype(np.float16)
    )

    df_means['rolling_sold_mean_28'] = (
        df_means.groupby(['item_id', 'dept_id', 'cat_id'])['sold']
        .transform(lambda x: x.rolling(window=28, min_periods=1).mean()).astype(np.float16)
    )

    df_means['expanding_sold_mean'] = (
        df_means.groupby(['item_id', 'dept_id', 'cat_id'])['sold']
        .transform(lambda x: x.expanding(2).mean()).astype(np.float16)
    )

    # Realiza las asignaciones asegurando que las dimensiones coincidan
    for col in ["rolling_sold_mean", "rolling_sold_mean_14", "rolling_sold_mean_28", "expanding_sold_mean"]:
        df.loc[df['d'] == day, col] = df_means.loc[df_means['d'] == day, col].reindex(df[df['d'] == day].index).values

    return df

In [ ]:
def predictions(df, duration,model, day_1):

    # Guarda las ventas reales en una nueva columna antes de sobrescribir 'sold' con predicciones
    if 'sold_reales' not in df.columns:
        df['sold_reales'] = df['sold']


    for day in range(day_1, day_1+duration+1):
        df = calculate_lags_and_means(df, day)
        X_day = df[df['d'] == day].drop(columns=['sold', "pred_sales","sold_reales"], errors='ignore')
        predictions = model.predict(X_day)
        predictions = np.round(predictions)
        df.loc[df['d'] == day, 'sold'] = predictions
    return df

# DEPLOY

In [ ]:
model=xgb.XGBRegressor()
model.load_model('/content/drive/MyDrive/ordenador/TEORÍA+PROYECTOS/m5-forecasting-accuracy/models/final_model.json')

In [ ]:
mb = modelbit.login()

In [ ]:
def shop_model_predict(df_entrada, day_1, duration):
  df_entrada = predictions(df_entrada, duration, model, day_1)
  return df_entrada


In [ ]:
mb.deploy(shop_model_predict)